# Google Analytics 4 (GA4) Data Analysis

This notebook connects to BigQuery to analyze GA4 event data.

**Project ID:** `analytics-473719`  
**Dataset ID:** `analytics_306941895`

In [ ]:
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt

# Set your Project ID and Dataset ID
PROJECT_ID = "analytics-473719"
DATASET_ID = "analytics_306941895"

# Initialize the BigQuery Client
# Ensure you have authenticated with `gcloud auth application-default login` 
# or set GOOGLE_APPLICATION_CREDENTIALS environment variable.
client = bigquery.Client(project=PROJECT_ID)

print(f"Connected to BigQuery Project: {PROJECT_ID}")

## Helper Function

In [ ]:
def run_query(sql):
    """Runs a BigQuery query and returns the result as a DataFrame."""
    print("Running query...")
    query_job = client.query(sql)
    df = query_job.to_dataframe()
    print(f"Query complete. Retrieved {len(df)} rows.")
    return df

## 1. Daily Event Count (Last 30 Days)
This query counts the total number of events for each day in the last 30 days.

In [ ]:
sql_daily_events = f"""
    SELECT
        event_date,
        COUNT(*) AS total_events
    FROM
        `{PROJECT_ID}.{DATASET_ID}.events_*`
    WHERE
        _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY))
        AND FORMAT_DATE('%Y%m%d', CURRENT_DATE())
    GROUP BY
        event_date
    ORDER BY
        event_date ASC
"""

df_daily = run_query(sql_daily_events)
df_daily.head()

In [ ]:
# Plotting the results
if not df_daily.empty:
    df_daily['event_date'] = pd.to_datetime(df_daily['event_date'])
    plt.figure(figsize=(12, 6))
    plt.plot(df_daily['event_date'], df_daily['total_events'], marker='o')
    plt.title('Daily Event Count (Last 30 Days)')
    plt.xlabel('Date')
    plt.ylabel('Total Events')
    plt.grid(True)
    plt.show()

## 2. Top 10 Events by Volume
Identify the most frequent event names.

In [ ]:
sql_top_events = f"""
    SELECT
        event_name,
        COUNT(*) AS event_count
    FROM
        `{PROJECT_ID}.{DATASET_ID}.events_*`
    WHERE
        _TABLE_SUFFIX BETWEEN FORMAT_DATE('%Y%m%d', DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY))
        AND FORMAT_DATE('%Y%m%d', CURRENT_DATE())
    GROUP BY
        event_name
    ORDER BY
        event_count DESC
    LIMIT 10
"""

df_top_events = run_query(sql_top_events)
df_top_events

## 3. Visualize Top Events

In [ ]:
if not df_top_events.empty:
    plt.figure(figsize=(10, 6))
    plt.barh(df_top_events['event_name'], df_top_events['event_count'], color='skyblue')
    plt.xlabel('Count')
    plt.title('Top 10 Events (Last 7 Days)')
    plt.gca().invert_yaxis()
    plt.show()